# Подготовка расчетной сетки для применения гидрогеологической модели

Задача: Имеется полигональный векторный слой, представляющий собой расчетную сетку для модели. Так как при моделировании взимное расположение ячеек сетки определяется на основе целочисленных индексов, а не координат, необходимо каждой ячейке присвоить индексы.

Размер расчетной матрицы 200 строк на 456 столбцов

In [1]:
# Импортируем нужные библиотеки
import numpy as np
import pandas as pd
import math
import fiona
import geopandas

Исходный файл представлен в формате shp.

In [4]:
shp_path = '/media/mikhail/Data/rotated_grid.shp'
shp = geopandas.read_file(shp_path)
# Переводим в датафрейм
dataset = pd.DataFrame(shp)
dataset.head(5)

,PKUID,ID,ROW,COL,x,y,geometry
0,1,1951,195,1,561316.472776,6.600035e+06,"POLYGON ((561311.935 6600029.855, 561311.049 6..."
1,2,1952,195,2,561326.433456,6.600036e+06,"POLYGON ((561321.896 6600030.741, 561321.010 6..."
2,3,1953,195,3,561336.394136,6.600037e+06,"POLYGON ((561331.857 6600031.627, 561330.971 6..."
3,4,1954,195,4,561346.354816,6.600038e+06,"POLYGON ((561341.817 6600032.513, 561340.932 6..."
4,5,1955,195,5,561356.315497,6.600039e+06,"POLYGON ((561351.778 6600033.399, 561350.892 6..."


Таким образом, каждая строчка в данном датафрейме - это ячейка матрицы расчетной сетки. Ключевыми столбцами в данном случае являются "x" и "y", которые представляют собой координаты центроидов ячеек. Проблема заключается в том, что матрица повернута под некоторым углом, из-за чего присвоить индексы ячейкам на основе исходных координат представляется крайне затруднительным. Более того, в виду особенностей записи полей в атрибутивную таблицу, соседние строки в датафрейме очень часто не являются соседними ячейками в расчетной сетке. 

Для решения задачи предложен следующий подход:
#### Необходимо найти угол Альфа, на который повернуть систему координат так, чтобы была получена система координат, пригодная для индексирования данной матрицы

Схему подхода можно увидеть ниже

![Rotation.png](https://raw.githubusercontent.com/Dreamlone/State_Hydrological_Institute/master/images/rotation.png)

Выпишем координаты вектора в исходной системе координат, который является "подошвой" матрицы

In [7]:
# Координаты нижнего левого угла:
x1 = 561317.35869400005
y1 = 6600025.31799000036
# Координаты нижнего правого угла:
x2 = 565520.76572000002
y2 = 6600399.17542000022

# Расчитываем координаты вектора в старой системе координат
x = x2 - x1
y = y2 - y1
print('Координата x вектора -', x)
print('Координата y вектора -', y)

Координата x вектора - 4203.407025999972
Координата y вектора - 373.85742999985814


Синус и косинус угла поворота

In [8]:
# Первое, что нужно сделать, рассчитать синус и косинус угла поворота альфа
# a и b - катеты
# c - гипотенуза
# ВНИМАНИЕ!!! Иcпользуются не координаты точки для определения угла поворота альфа, а КООРДИНАТЫ ВЕКТОРА
def coeff(a,b,c):
    # Косинус - отношение прилежащего катета к гипотенузе
    cosA = a/c
    # Синус - отношение противолежащего катета к гипотенузе
    sinA = b/c
    return(cosA, sinA)

cosA, sinA = coeff(a = x, b = y, c = math.sqrt(x**2 + y**2))
print('Значение синуса угла поворота альфа -', sinA)
print('Значение косинуса угла поворота альфа -', cosA)

Значение синуса угла поворота альфа - 0.08859180804641135
Значение косинуса угла поворота альфа - 0.9960680155225686


Поступим несколько расточительно в вычислительном плане, но все же: подготовим функцию решения системы линейных уравнений

In [9]:
# Теперь, переведем старые координаты в новые
# Решаем систему уравнений, то есть находим новые координаты
def solve_equation(sinA, cosA, x , y):
    a = np.array([[cosA, -sinA], [sinA, cosA]])
    answers = np.array([x, y])
    new_coords = np.linalg.solve(a, answers)
    # Массив [координата x, координата y]
    return(new_coords)

### Переход к новой системе координат

In [10]:
Xs = []
Ys = []
# Пройдемся по всем строкам таблицы
for i in np.array(dataset[['x','y']]):
    old_x = i[0]
    old_y = i[1]
    new = solve_equation(sinA, cosA, x = old_x, y = old_y)
    new_x = new[0]
    new_y = new[1]
    # Записываем в список
    Xs.append(new_x)
    Ys.append(new_y)

# Добавляем новые столбцы в датасет
dataset['New_X'] = Xs
dataset['New_Y'] = Ys
dataset.head(5)

,PKUID,ID,ROW,COL,x,y,geometry,New_X,New_Y
0,1,1951,195,1,561316.472776,6.600035e+06,"POLYGON ((561311.935 6600029.855, 561311.049 6...",1.143818e+06,6.524356e+06
1,2,1952,195,2,561326.433456,6.600036e+06,"POLYGON ((561321.896 6600030.741, 561321.010 6...",1.143828e+06,6.524356e+06
2,3,1953,195,3,561336.394136,6.600037e+06,"POLYGON ((561331.857 6600031.627, 561330.971 6...",1.143838e+06,6.524356e+06
3,4,1954,195,4,561346.354816,6.600038e+06,"POLYGON ((561341.817 6600032.513, 561340.932 6...",1.143848e+06,6.524356e+06
4,5,1955,195,5,561356.315497,6.600039e+06,"POLYGON ((561351.778 6600033.399, 561350.892 6...",1.143858e+06,6.524356e+06


### Индексирование

Теперь стороны прямоугольной расчетной сетки перпендикулярны координатным осям (или совпадают с ними) и можно начинать присваивать индексы. Мы будем двигаться итеративно, начиная слева двигаться вправо, присваивая индексы строк и сверху вних, присваивая индексы столбцов.

In [12]:
# Теперь представим следующую ситуацию, у нас есть 456 столбцов по 200 строк в каждом
# Цикл индексации столбцов
# Нумерация начинается с 1
for element in range(1, 457):
    x_small = dataset.nsmallest(200, 'New_X')

    # Теперь установим индексы и заменим уже пройденные значения на Nan
    for i in list(x_small.index):
        dataset['COL'][dataset.index == i] = element
        dataset['New_X'][dataset.index == i] = np.nan

In [14]:
# Цикл индексации строк (200 строк по 456 столбцов в каждой)
# Нумерация начинается с 1
for element in range(200, 0, -1):
    y_small = dataset.nsmallest(456, 'New_Y')

    # Теперь установим индексы и заменим уже пройденные значения на Nan
    for i in list(y_small.index):
        dataset['ROW'][dataset.index == i] = element
        dataset['New_Y'][dataset.index == i] = np.nan

# Посмотрим на то, что в итоге получилось        
dataset.head(5)

,PKUID,ID,ROW,COL,x,y,geometry,New_X,New_Y
0,1,1951,199,1,561316.472776,6.600035e+06,"POLYGON ((561311.935 6600029.855, 561311.049 6...",NaN,NaN
1,2,1952,199,2,561326.433456,6.600036e+06,"POLYGON ((561321.896 6600030.741, 561321.010 6...",NaN,NaN
2,3,1953,199,3,561336.394136,6.600037e+06,"POLYGON ((561331.857 6600031.627, 561330.971 6...",NaN,NaN
3,4,1954,199,4,561346.354816,6.600038e+06,"POLYGON ((561341.817 6600032.513, 561340.932 6...",NaN,NaN
4,5,1955,199,5,561356.315497,6.600039e+06,"POLYGON ((561351.778 6600033.399, 561350.892 6...",NaN,NaN


In [15]:
# Добавим расчитанные индексы
shp['ROW'] = dataset['ROW']
shp['COL'] = dataset['COL']

# Сохраним все в файл
shp.to_file("/media/mikhail/Data/table.shp")